In [40]:
from collections import defaultdict
import doctest
from functools import cache
from itertools import combinations, islice, permutations

In [97]:
def permutation_id(n):
    return ''.join(sorted(str(n)))

def is_permutation_of(m, n):
    """
    >>> is_permutation_of(1, 1)
    True
    >>> is_permutation_of(1, 10)
    False
    >>> is_permutation_of(404, 440)
    True
    >>> is_permutation_of(41063625, 56623104)
    True
    """
    return permutation_id(m) == permutation_id(n)

@cache
def cubes_upto(upto):
    """
    >>> sorted(cubes_upto(5))
    [1, 8, 27, 64, 125]
    """
    return {n**3 for n in range(1, upto + 1)}

def make_fast_is_cubic(cache_cubes_upto=999):
    cubes = {str(c) for c in cubes_upto(cache_cubes_upto)}
    return lambda n: n in cubes

fast_is_cubic = make_fast_is_cubic()

def fast_permutations(n):
    for m in permutations(permutation_id(n)):
        if m[0] != '0':
            yield ''.join(m)

@cache
def cubic_permutations(n):
    """
    >>> sorted(cubic_permutations(56623104))
    ['41063625', '56623104', '66430125']
    """
    return set(m for m in fast_permutations(n) if fast_is_cubic(m))

In [42]:
def find_n_with_m_permutations(m, consider_cubes_upto=9999):
    for i in cubes_upto(consider_cubes_upto):
        ps = cubic_permutations(i)
        if len(ps) == m:
            return (i, ps)

In [37]:
%%time
find_n_with_m_permutations(3)

Wall time: 3 ms


(56623104, {'41063625', '56623104', '66430125'})

In [38]:
%%time
# Basically too slow
#find_n_with_m_permutations(5)

Wall time: 0 ns


In [124]:
def fast_find_n_with_p_permutations(p, consider_cubes_upto=9999):
    permutations_by_id = defaultdict(set)
    for c in cubes_upto(consider_cubes_upto):
        permutations_by_id[permutation_id(c)].add(c)
    return min((min(ps) for ps in permutations_by_id.values() if len(ps) == p), default=None)

def faster_find_n_with_p_permutations(p, consider_min_perm_length, consider_cubes_upto=9999):
    """To consider by increasing permutation length instead."""
    cubes_by_length = defaultdict(set)
    for cube in cubes_upto(consider_cubes_upto):
        cubes_by_length[len(str(cube))].add(cube)
    
    for length, cubes in cubes_by_length.items():
        if length < consider_min_perm_length: continue
        #print(f'Trying permutations of length {length} from {len(cubes)} cubes')
        permutations_by_id = defaultdict(set)
        for c in cubes:
            permutations_by_id[permutation_id(c)].add(c)
        solution = min((min(ps) for ps in permutations_by_id.values() if len(ps) == p), default=None)
        if solution is not None: return solution

In [125]:
%%time
fast_find_n_with_p_permutations(3)

Wall time: 20 ms


41063625

In [127]:
%%time
fast_find_n_with_p_permutations(5)

Wall time: 20 ms


127035954683

In [126]:
%%timeit -n 10
fast_find_n_with_p_permutations(5)

16.8 ms ± 423 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [123]:
%%timeit -n 10
# Ok, so actually it's slower...
faster_find_n_with_p_permutations(5, consider_min_perm_length=11)

17 ms ± 637 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [98]:
doctest.testmod(verbose=True)

Trying:
    sorted(cubes_upto(5))
Expecting:
    [1, 8, 27, 64, 125]
ok
Trying:
    sorted(cubic_permutations(56623104))
Expecting:
    ['41063625', '56623104', '66430125']
ok
Trying:
    is_permutation_of(1, 1)
Expecting:
    True
ok
Trying:
    is_permutation_of(1, 10)
Expecting:
    False
ok
Trying:
    is_permutation_of(404, 440)
Expecting:
    True
ok
Trying:
    is_permutation_of(41063625, 56623104)
Expecting:
    True
ok
8 items had no tests:
    __main__
    __main__.fast_find_n_with_m_permutations
    __main__.fast_find_n_with_p_permutations
    __main__.fast_is_cubic
    __main__.fast_permutations
    __main__.find_n_with_m_permutations
    __main__.make_fast_is_cubic
    __main__.permutation_id
3 items passed all tests:
   1 tests in __main__.cubes_upto
   1 tests in __main__.cubic_permutations
   4 tests in __main__.is_permutation_of
6 tests in 11 items.
6 passed and 0 failed.
Test passed.


TestResults(failed=0, attempted=6)